In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import time
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

import chainer
import chainer.functions as F
import chainer.links as L
from chainer import optimizers
from chainer import initializers
#from chainer import serializers
import  pickle # hdf5のインストールがめんどいからpickleでシリアライズ

from dataset import load_dataset

In [2]:
# cifar10のロード、前処理
x_train,y_train,x_test,y_test = load_dataset()
x_train = x_train.astype(np.float32).transpose((0, 3, 1, 2))
x_test = x_test.astype(np.float32).transpose((0, 3, 1, 2))
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)

# MLPのため
x_train_mlp = x_train.reshape((50000, 32*32*3))
x_test_mlp = x_test.reshape((10000, 32*32*3))

# Hello CNN

In [3]:
class Cifar10(chainer.Chain):

    def __init__(self):
        super(Cifar10, self).__init__(
            raise NotImplementedError()
        )
        
    def __call__(self, x):
        raise NotImplementedError()


In [4]:
class MnistMLP(chainer.Chain):
    def __init__(self, n_units):
        super(MnistMLP, self).__init__(
            l1=L.Linear(None, n_units),
            l2=L.Linear(n_units, n_units),
            l3=L.Linear(n_units, 10),
        )

    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        return self.l3(h2)

In [5]:
def train(model, optimizer, x_data, y_data, batchsize=10):
    N = x_data.shape[0] # 学習データ数(63000)
    perm = np.random.permutation(N) # 学習する順番をランダムに入れ替え
    
    sum_accuracy = 0    # 累計正答率
    sum_loss = 0        # 累計誤差
    start = time.time() # 開始時刻
    
    # batchsize個ずつ学習
    for i in tqdm(range(0, N, batchsize)):
        x = chainer.Variable(np.asarray(x_data[i:i + batchsize]))
        t = chainer.Variable(np.asarray(y_data[i:i + batchsize]))

        # パラメータの更新(学習)
        optimizer.update(model, x, t)
        
        sum_loss += float(model.loss.data) * len(t.data)
        sum_accuracy += float(model.accuracy.data) * len(t.data)
        
    end = time.time() # 終了時刻
    elapsed_time = end - start
    throughput = N / elapsed_time # 単位時間当たりの作業量
    print("train mean loss={}, accuracy={}, throughput={} image/sec".format(
        sum_loss / N, sum_accuracy / N, throughput))

In [6]:
def evaluate(model, x_data, y_data, batchsize=10):
    N = x_data.shape[0] # テストデータ数(7000)
    perm = np.random.permutation(N) # 学習する順番をランダムに入れ替え
    
    sum_accuracy = 0    # 累計正答率
    sum_loss = 0        # 累計誤差
    
    # batchsize個ずつ評価
    for i in tqdm(range(0, N, batchsize)):
        # 評価の時はvolatile
        x = chainer.Variable(np.asarray(x_data[i:i + batchsize]))
        t = chainer.Variable(np.asarray(y_data[i:i + batchsize]))
        
        # 評価
        loss = model(x, t)
        sum_loss += float(loss.data) * len(t.data)
        sum_accuracy += float(model.accuracy.data) * len(t.data)
        
    print("test mean loss={}, accuracy={}".format(
        sum_loss / N, sum_accuracy / N))

In [7]:
cifar = L.Classifier(Cifar10())
optimizer = optimizers.Adam()
optimizer.setup(cifar)

In [8]:
# train CNN
n_epoch = 10 # 学習回数(学習データを何周するか)
for epoch in range(1, n_epoch + 1):
    print("epoch", epoch)
    
    # 訓練
    train(cifar, optimizer, x_train, y_train, batchsize=100)
    
    # 評価
    evaluate(cifar, x_test, y_test, batchsize=100)

  0%|          | 0/500 [00:00<?, ?it/s]

epoch 1


  0%|          | 0/100 [00:00<?, ?it/s]

train mean loss=3.234859844207764, accuracy=0.2831200006902218, throughput=91.88863916396144 image/sec


  0%|          | 0/500 [00:00<?, ?it/s]

...

test mean loss=1.2073948550224305, accuracy=0.5911999982595444
epoch 10


  0%|          | 0/100 [00:00<?, ?it/s]

train mean loss=1.097481538295746, accuracy=0.613839998781681, throughput=87.65032667122092 image/sec


100%|██████████| 100/100 [00:54<00:00,  1.83it/s]

test mean loss=1.2174324989318848, accuracy=0.587099997997284


In [9]:
mlp = L.Classifier(MnistMLP(1000))
optimizer = optimizers.Adam()
optimizer.setup(mlp)

In [10]:
# train MLP
n_epoch = 10 # 学習回数(学習データを何周するか)
for epoch in range(1, n_epoch + 1):
    print("epoch", epoch)
    
    # 訓練
    train(mlp, optimizer, x_train_mlp, y_train, batchsize=100)
    
    # 評価
    evaluate(mlp, x_test_mlp, y_test, batchsize=100)

  0%|          | 0/500 [00:00<?, ?it/s]

epoch 1


 10%|█         | 10/100 [00:00<00:00, 95.19it/s]

train mean loss=31.926015453100206, accuracy=0.2309799997843802, throughput=1674.806793054277 image/sec


  0%|          | 2/500 [00:00<00:32, 15.46it/s]

...

test mean loss=2.2844183921813963, accuracy=0.17700000055134296
epoch 10


 24%|██▍       | 24/100 [00:00<00:00, 106.78it/s]

train mean loss=2.164412490129471, accuracy=0.1722800000086427, throughput=635.4210417185914 image/sec


100%|██████████| 100/100 [00:00<00:00, 123.59it/s]

test mean loss=2.292631440162659, accuracy=0.1077999991737306


# Hello ResNet

In [11]:
class ResBlock(chainer.Chain):

    def __init__(self, in_size, ch):
        super(ResBlock, self).__init__()
        initialW = initializers.HeNormal()
        
        with self.init_scope():
            raise NotImplementedError()
            self.conv1 = L.Convolution2D(
                in_size, ch, 1, 1, 0, initialW=initialW, nobias=True)
            self.bn1 = L.BatchNormalization(ch)
            self.conv2 = L.Convolution2D(
                ch, ch, 3, 1, 1, initialW=initialW, nobias=True)
            self.bn2 = L.BatchNormalization(ch)
            self.conv3 = L.Convolution2D(
                ch, in_size, 1, 1, 0, initialW=initialW, nobias=True)
            self.bn3 = L.BatchNormalization(in_size)

    def __call__(self, x):
        h = F.relu(self.bn1(self.conv1(x)))
        h = F.relu(self.bn2(self.conv2(h)))
        h = self.bn3(self.conv3(h))

        return F.relu(h + x)

class ResNetSmall(chainer.Chain):

    def __init__(self):
        super(ResNetSmall, self).__init__(
            conv1 = L.Convolution2D(
                3, 32, 5, initialW=initializers.HeNormal()),
            bn1 = L.BatchNormalization(32),
            res1= ResBlock(32, 32),
            res2= ResBlock(32, 32),
            res3= ResBlock(32, 32),
            fc=L.Linear(None, 10),
        )
        
    def __call__(self, x):
        h = self.bn1(self.conv1(x))
        h = F.max_pooling_2d(F.relu(h), 3, stride=2)
        h = self.res2(h)
        h = self.res3(h)
        h = F.average_pooling_2d(h, 7, stride=1)
        return self.fc(h)

In [12]:
resnet = L.Classifier(ResNetSmall())
optimizer = optimizers.Adam()
optimizer.setup(resnet)

In [13]:
n_epoch = 10 # 学習回数(学習データを何周するか)
for epoch in range(1, n_epoch + 1):
    print("epoch", epoch)
    
    # 訓練
    train(resnet, optimizer, x_train, y_train, batchsize=100)
    
    # 評価
    evaluate(resnet, x_test, y_test, batchsize=100)

  0%|          | 0/500 [00:00<?, ?it/s]

epoch 1


  0%|          | 0/100 [00:00<?, ?it/s]

train mean loss=1.4206987286806108, accuracy=0.4871799971908331, throughput=132.5744042418934 image/sec


  0%|          | 0/500 [00:00<?, ?it/s]

...

test mean loss=0.802440499663353, accuracy=0.7250000023841858
epoch 10


  0%|          | 0/100 [00:00<?, ?it/s]

train mean loss=0.6727387962937355, accuracy=0.7680999985933303, throughput=130.6262769361969 image/sec


100%|██████████| 100/100 [00:39<00:00,  2.61it/s]

test mean loss=0.7964427852630616, accuracy=0.7275000023841858
